In [1]:
import pandas as pd
from functools import reduce
import warnings
from statsmodels.tsa.vector_ar import vecm
import numpy as np

In [2]:
spread_data = pd.read_csv("spread_data.csv", index_col="dtime")
SFR = pd.read_csv("SFRU24.csv", index_col="dtime")
SFR_trades = pd.read_csv("SFRU24_trades.csv", index_col="dtime")
FF = pd.read_csv("FFV24.csv", index_col="dtime")
FF_trades = pd.read_csv("FFV24_trades.csv", index_col="dtime")

In [3]:
FF

,instrument,bid_price,bid_size,bid_orders,ask_price,ask_size,ask_orders,rectime,quote_id,nanos
dtime,,,,,,,,,,
2024-03-12 12:27:00.000037+00:00,FFV24,95.22,350,7,95.23,686,14,2024-03-12 12:27:00.010102+00:00,3178376,0
2024-03-12 12:27:00.000798+00:00,FFV24,95.22,350,7,95.23,686,14,2024-03-12 12:27:00.010102+00:00,3178376,0
2024-03-12 12:27:00.007363+00:00,FFV24,95.22,350,7,95.23,686,14,2024-03-12 12:27:00.010102+00:00,3178376,0
2024-03-12 12:27:00.008031+00:00,FFV24,95.22,350,7,95.23,686,14,2024-03-12 12:27:00.010102+00:00,3178376,0
2024-03-12 12:27:00.022696+00:00,FFV24,95.22,350,7,95.23,686,14,2024-03-12 12:27:00.010102+00:00,3178376,0
...,...,...,...,...,...,...,...,...,...,...
2024-03-12 12:33:59.997391+00:00,FFV24,95.23,12,1,95.24,233,9,2024-03-12 12:34:00.005392+00:00,3354585,997000000
2024-03-12 12:33:59.997427+00:00,FFV24,95.23,12,1,95.24,233,9,2024-03-12 12:34:00.005392+00:00,3354585,997000000
2024-03-12 12:33:59.998001+00:00,FFV24,95.23,12,1,95.24,233,9,2024-03-12 12:34:00.005392+00:00,3354585,997000000


In [4]:
SFR
#We just want first 5000 rows of SOFR and FF
SFR = SFR.iloc[5000:15000]
FF = FF.iloc[5000:15000]


In [5]:
class InfoShares:
    """
    Reference From: https://rdrr.io/rforge/ifrogs/src/R/pdshare.R
    """
    def __init__(self, data, k_ar_diff, maxLag=None, deterministic="ci"):
        self.y_t = data
        self.endog_names = data.columns
        self.k_ar_diff = k_ar_diff
        self.deterministic = deterministic

        if isinstance(k_ar_diff, int) and k_ar_diff <= 0:
            raise Exception("k_ar_diff must be greater than 0")

        elif isinstance(k_ar_diff, str):
            if not k_ar_diff in ['aic', 'bic', 'hqic', 'fpe']:
                raise Exception("k_ar_diff must be in ['aic', 'bic', 'hqic', 'fpe']")
            else:
                if not isinstance(maxLag, int):
                    raise Exception("maxLag must be Integer")
                self.k_ar_diff = self.select_k_ar_diff(maxLag)

        self.neqs = data.shape[1]

        if not self.neqs == 2:
            raise Exception("only support 2 variable series")

    def select_k_ar_diff(self, maxLag):
        res = vecm.select_order(self.y_t, maxlags=maxLag, deterministic=self.deterministic)
        k_ar_diff = res.__getattribute__(self.k_ar_diff)

        k_ar_diff += 1 if k_ar_diff == 0 else k_ar_diff
        return k_ar_diff

    def fit(self):
        alpha, beta, gamma, omega, corrcoefs = self.vecm()

        def _orth(vector):
            v = np.array(vector).flatten()
            return np.matrix([-1 * v[1], v[0]]).T

        orth_a = _orth(alpha)
        orth_b = _orth(beta)
        g = reduce(lambda x, y: np.add(x, y), gamma)
        pi = orth_a.T.dot(np.eye(len(alpha)) - g).dot(orth_b) ** (-1)
        psi = orth_b.dot(pi).dot(orth_a.T)

        if not np.allclose(np.linalg.det(psi), 0):
            warnings.warn("Financial might not convergence")

        s1 = ishares(psi[0], omega)
        s2 = mishares(corrcoefs, psi[0], omega)
        cs = compshares(orth_a)

        results = {"infoShares": s1, "MInfoShares": s2, "compShares": cs,
                   "k": self.k_ar_diff, "a": alpha, "b": beta, "g": gamma,
                   "omega": omega, "corrcoefs": corrcoefs}

        return ResultsWarper(results)

    def get_clean_results(self, res):
        alpha = res.alpha
        beta = res.beta
        gamma = res.gamma

        u = res.resid
        gamma = np.array(np.hsplit(gamma, self.k_ar_diff))

        omega = np.cov(u, rowvar=False)
        corrcoefs = np.corrcoef(u, rowvar=False)

        return alpha, beta, gamma, omega, corrcoefs

    def vecm(self):
        k_ar_diff = self.k_ar_diff
        from statsmodels.tsa.vector_ar.vecm import VECM
        coint_rank = self.neqs - 1
        model = VECM(self.y_t, k_ar_diff=k_ar_diff, deterministic=self.deterministic, coint_rank=coint_rank)
        res = model.fit()
        return self.get_clean_results(res)

class ResultsWarper:
    def __init__(self, kwargs):
        for key, val in kwargs.items():
            setattr(self, key, val)

def compshares(orth_a):
    abs_ = abs(orth_a)
    return abs_ / abs_.sum()

def ishares(psi, omega):
    total_variance = psi.dot(omega).dot(psi.T)
    f = np.linalg.cholesky(omega)
    proportion = np.power(psi.dot(f), 2) / total_variance
    return proportion

def mishares(corr, psi, omega):
    from scipy.linalg import eigh
    _lambda, evec = eigh(corr)
    _lambda = np.diag(_lambda)
    g = evec.transpose()
    v = np.diag(np.sqrt(np.diag(omega)))
    matrix_f_asterisk = np.dot(g.dot(np.power(np.linalg.inv(_lambda), .5)), g.transpose()).dot(np.linalg.inv(v))
    matrix_f_asterisk = np.linalg.inv(matrix_f_asterisk)
    phi_asterisk = psi.dot(matrix_f_asterisk)
    total_variance = psi.dot(omega).dot(psi.T)
    proportion_mis = np.power(phi_asterisk, 2) / total_variance
    return proportion_mis


In [6]:
def compute_statistic(spread_data, leg1_quotes, leg1_trades, leg2_quotes, leg2_trades ):
    # We want to create a datafame with the bid price column of the leg1_trades, bid column of leg_2.
    leg1_quotes_bid = leg1_quotes["bid_price"]
    leg1_quotes_bid=leg1_quotes_bid.to_frame()
    leg2_quotes_bid = leg2_quotes["bid_price"]
    leg2_quotes_bid=leg2_quotes_bid.to_frame()
    merged_df = pd.merge(leg1_quotes_bid, leg2_quotes_bid, how='outer', left_index=True, right_index=True)
    synchronized_df = merged_df.ffill()
    synchronized_df = synchronized_df.bfill() #First NAN values filled with the first value of the series.
    synchronized_df = synchronized_df.reset_index(drop=True)#Reseting index for IS.
    #We change the type of the dataframe to float32 to avoid any type errors
    synchronized_df=synchronized_df.astype('float32')
    info_shares = InfoShares(synchronized_df, k_ar_diff=1)#We assume there exists cointegration relationship.
    results_1 = info_shares.fit()
    synchronized_df_2= synchronized_df[['bid_price_y','bid_price_x']]
    info_shares_2 = InfoShares(synchronized_df_2, k_ar_diff=1)
    results_2 = info_shares_2.fit()

    return results_1.infoShares, results_2.infoShares

In [7]:
compute_statistic(spread_data, SFR, SFR_trades, FF, FF_trades )

(matrix([[0.53235322, 0.46764678]]), matrix([[0.47672627, 0.52327373]]))

In [8]:
import statsmodels as sm
print(sm.__version__)

0.14.1
